In [49]:
%pip install spacy_thai
%pip install gensim

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
     ---------------------------------------- 0.1/24.4 MB 1.3 MB/s eta 0:00:19
     ---------------------------------------- 0.2/24.4 MB 2.3 MB/s eta 0:00:11
      --------------------------------------- 0.5/24.4 MB 3.3 MB/s eta 0:00:08
     - -------------------------------------- 1.0/24.4 MB 5.1 MB/s eta 0:00:05
     -- ------------------------------------- 1.5/24.4 MB 6.5 MB/s eta 0:00:04
     --- ------------------------------------ 2.0/24.4 MB 7.3 MB/s eta 0:00:04
     ---- ----------------------------------- 2.7/24.4 MB 8.1 MB/s eta 0:00:03
     ----- ---------------------------------- 3.2/24.4 MB 8.4 MB/s eta 0:00:03
     ------ --------------------------------- 3.7/24.4 MB 8.9 MB/s eta 0:00:03
     ------- -------------------------------- 4.4/24.4 MB 9.3 MB/s eta 0:00:03
     -------- ------------------------------- 4.9/24.4 MB 9.6 MB/s eta 0:00:03
     -------- ------------------------------- 5.5/24.4 MB 


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install pythainlp
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [15 lines of output]
  The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  rather than 'sklearn' for pip commands.
  
  Here is how to fix this error in the main use cases:
  - use 'pip install scikit-learn' rather than 'pip install sklearn'
  - replace 'sklearn' by 'scikit-learn' in your pip requirements files
    (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
  - if the 'sklearn' package is used by one of your dependencies,
    it would be great if you take some time to track which package uses
    'sklearn' instead of 'scikit-learn' and report it to their issue tracker
  - as a last resort, set the environment variable
    SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
  
  More information is available at
  https://github.com/scikit-learn/sklearn-pypi-package
  [end of output]
  
  note: This error originates from a subpr

In [7]:
import numpy as np 
import pandas as pd

In [8]:
df = pd.read_csv(r'C:\Users\User\OneDrive\Desktop\seen_jupyter\research\prachatai(TK Edition).csv')
print(df.shape)

(6721, 1)


In [ ]:
from pythainlp.tokenize import word_tokenize
# df["tokens_200"] = df["0"].astype(str).apply(
#     lambda x: word_tokenize(x[:200])
# )
# df

tokens_200_row0 = word_tokenize(str(df.loc[0, "0"])[:200])
# tokens_200_row0

In [55]:
from pythainlp.util import normalize
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_stopwords

stop = set(thai_stopwords())

# sample text row 0 colum "0" 
list_txt =  df["0"].fillna("").astype(str).head(2).tolist() #["...", "...", "..."] ลด NaN
# list_txt =  df["0"].fillna("").astype(str).tolist() #["...", "...", "..."] ลด NaN


# def preprocessing(list_text):
#     # normalize for thai text ( ex. ำ  = \u0e33 and normalize to real ำ )
#     normalized_txt_list = [normalize(text) for text in list_text]
#     # print(len(normalized_txt_list))

#     # word tokenize 
#     tokens_text_list = [word_tokenize(text, engine="newmm") for text in normalized_txt_list]
#     # print(tokens_text_list[0][:30]) # [["...","..."], ["...","..."] ...]

#     # stopwords + filter length > 1
#     tokens_clean_list = [[t for t in tokens_text_sml_list if t.strip() and t not in stop and len(t) > 1]  for tokens_text_sml_list in tokens_text_list]
#     # txt, tokens_clean

#     # put word in list join to string and separate by space
#     tokens_clean_str = [' '.join(tokens_clean) for tokens_clean in tokens_clean_list]
#     # print(tokens_clean_str)

#     return tokens_clean_str

list_txt


['โดย ดร.เจิมศักดิ์\xa0 ปิ่นทอง\n\n\xa0\n\n\xa0\n\nข้อเขียนชิ้นนี้ เป็นการพูดกันอย่างตรงไปตรงมาที่สุด\n\n\xa0\n\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 ข้อเท็จจริงของบ้านเมือง ปรากฏดังต่อไปนี้\n\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 ๑) ขณะนี้ มีปัญญาชน (คนที่มี "ปัญญา" ในสังคม) จำนวนมาก ซึ่งเข้าถึงข้อมูล ข่าวสาร และสนใจติดตามเรียนรู้ความเป็นไปของบ้านเมือง ไม่ว่าจะเป็น นิสิตนักศึกษา นักวิชาการ ครู อาจารย์ รวมไปถึง "ชนชั้นกลาง" ที่มีลักษณะปัจเจกชนสูง ได้พร้อมใจกันออกมาขับไล่ "ทักษิณ ออกไป"\n\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 ๒) ประเด็นสำคัญในการขับไล่ เกี่ยวข้องกับการขาดจริยธรรมร้ายแรง หลบเลี่ยงกฎกติกาสังคม ขาดความถูกต้องชอบธรรม \xa0"ทักษิณ" หากเป็นพระ ก็กำลังถูกพระในวัดที่มีความรู้ความสามารถ ขับไล่ "ทักษิณ ปราชิกออกไป" \n\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 \xa0๓) "ทักษิณ" ประกาศว่าจะไม่ออก (ไม่สึก) แต่จะขอประชามติว่าจะแก้รัฐธรรมนูญ เป็นการโยนความผิดให้รัฐธรรมนูญ\xa0 เสมือนพระถูกขับไล่ "ปราชิก" แต่กลับเลี่ยงไปถามว่า จะไปแก้ "วินัยสงฆ์" ไหม? \n\n\xa0\x

In [47]:
import re
from pythainlp.util import normalize
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_stopwords

stop = set(thai_stopwords())

# --- กรอง token ให้อยู่ในรูปที่ model ใช้ง่าย ---
_token_ok = re.compile(r"^[ก-๙a-zA-Z0-9]+$")

def clean_text(s: str) -> str:
    s = "" if s is None else str(s)
    s = s.replace("\xa0", " ").replace("\t", " ").replace("\r", " ").replace("\n", " ")
    s = normalize(s)
    s = s.replace('"', " ").replace('”', " ").replace("''", " ")   
    # ยุบช่องว่างหลายๆอันให้เหลือช่องว่างเดียว
    s = re.sub(r"\s+", " ", s).strip()
    return s
     

def preprocessing(list_text, engine="newmm", min_len=2, keep_num=True):
    tovec_list = []
    tfidf_list = []
    for text in list_text:
        text = clean_text(text)
        tokenized_data = word_tokenize(text, engine=engine)
        cleaned_text = []
        for tokenized_word in tokenized_data:
            tokenized_word = tokenized_word.strip()

            if not tokenized_word:
                continue
            if len(tokenized_word) < min_len:
                continue
            if tokenized_word in stop:
                continue
            if not _token_ok.match(tokenized_word):
                continue
            if not keep_num and tokenized_word.isdigit():
                continue
            cleaned_text.append(tokenized_word)

        tovec_list.append(cleaned_text)
        tfidf_list.append(" ".join(cleaned_text))
    return tovec_list, tfidf_list



In [46]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    ngram_range=(1,2),   # unigram, bigram 
    min_df=2 # at least appear in 2 documents
)

tovec_list, tfidf_list = preprocessing(list_txt)
# print(tfidf_list)
outcome = vectorizer.fit_transform(tfidf_list)
print(outcome.shape)
sample = outcome[:100]
outcome_df = pd.DataFrame(sample.toarray(), columns=vectorizer.get_feature_names_out())
outcome_df


['เจิม ศักดิ์ ปิ่น ทอง ข้อเขียน ชิ้น ตรงไปตรงมา ข้อเท็จจริง บ้านเมือง ดังต่อไปนี้ ปัญญาชน คน ปัญญา สังคม จำนวนมาก เข้าถึง ข้อมูล ข่าวสาร สนใจ ติดตาม เรียนรู้ ความเป็นไป บ้านเมือง นิสิตนักศึกษา นักวิชาการ ครู อาจารย์ รวมไปถึง ชนชั้นกลาง ลักษณะ ปัจเจกชน พร้อมใจกัน ออกมา ขับไล่ ทักษิณ ประเด็นสำคัญ การขับไล่ จริยธรรม ร้ายแรง หลบเลี่ยง กฎ กติกา สังคม ความถูกต้อง ชอบธรรม ทักษิณ พระ พระ วัด มีความรู้ ความสามารถ ขับไล่ ทักษิณ ราชิ กอ อก ทักษิณ ประกาศ สึก ประชามติ แก้ รัฐธรรมนูญ โยนความผิด รัฐธรรมนูญ เสมือน พระ ขับไล่ ราชิ แต่กลับ เลี่ยง ถาม แก้ วินัยสงฆ์ ไหม ทักษิณ จริยธรรม บกพร่อง แก้ รัฐธรรมนูญ ช่วยไม่ได้ ความคิดเห็น ทำ ประชามติ วันที่ ๑๙ เมษายน ๒๕๔๙ แก้ รัฐธรรมนูญ ไหม เลือก กระ ทำได้ ทำ ประชามติ ชัดเจน แก้ รัฐธรรมนูญ ประเด็น ใด ประกาศ ลงมติ ๙๐ ๑๒๐ ข้อมูล ด้านใน ประเด็น ที่จะ เห็นด้วย เห็นด้วย ทำงาน เต็มที่ มติ ประชาชน ผูกพัน คณะรัฐมนตรี น่าคิด ทักษิณ ออกมา สะท้อน รู้ กฎหมาย รัฐธรรมนูญ นึก ทำ กฎหมาย วิ่ง สั่งการ เนติ บริกร บิดเบือน บอ ทำ วันที่ ๑๙ เมษายน ดี ประชามติ ประชาพิจารณ์ บิดเบือน ประ

,10,13,16,19,21,60,of,กก,กการเม,กการเม อง,...,ให,ให การ,ได,ไทย,ไทย เร,ไม,ไร,ไล,ไว,ไหม
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.029575,0.042339,0.029575,0.072834,0.000000,0.000000,0.000000,0.000000,0.042339,0.084678
1,0.000000,0.000000,0.075765,0.000000,0.086601,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.049658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.010864,0.021727,0.009504,0.010864,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.037944,0.032591,0.007589,0.043606,0.010864,0.010864,0.016900,0.010864,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.050810,0.000000,0.000000,0.000000,0.000000,...,0.035493,0.000000,0.000000,0.029135,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.076697,0.038348,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.021990,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.018072,0.061968,0.000000,0.020656,0.000000,0.020656,0.020656,0.020656,...,0.028858,0.000000,0.043287,0.011845,0.000000,0.000000,0.016067,0.020656,0.041312,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071587,0.035794,0.000000,0.000000,...,0.025003,0.000000,0.050006,0.000000,0.000000,0.035794,0.027841,0.000000,0.000000,0.035794
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.087562,0.000000,0.087562,0.087562,...,0.000000,0.000000,0.000000,0.066947,0.029187,0.000000,0.022703,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.025182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.092323,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
# Thai2Vec
from pythainlp.word_vector import WordVector

class Thai2VecEncoder:
    def __init__(self):
        # โหลด pretrained model แค่ครั้งเดียว
        self.wv = WordVector(model_name="thai2fit_wv").get_model()
        self.dim = self.wv.vector_size

    def doc_vector(self, tokens: list):
        """
        tokens: list[str] ของข่าว 1 ชิ้น
        return: vector (dim,)
        """
        vecs = []

        for tok in tokens:
            if tok in self.wv:
                vecs.append(self.wv[tok])

        if not vecs:
            return np.zeros(self.dim, dtype=np.float32)

        return np.mean(vecs, axis=0).astype(np.float32) # เฉลี่ยตามเเกนในเเนวตั้ง 

    def transform(self, tovec_list: list):
        """
        tovec_list: list[list[str]]
        return: X shape (n_docs, dim)
        """
        return np.vstack([self.doc_vector(tokens) for tokens in tovec_list])
    
    def oov_rate(self, tovec_list, wv_model):
        total = 0
        oov = 0
        for toks in tovec_list:
            for t in toks:
                total += 1
                if t not in wv_model:
                    oov += 1
        return (oov / total) if total else 0.0

# ได้จาก preprocessing ของคุณ
tovec_list, tfidf_list = preprocessing(list_txt)

encoder = Thai2VecEncoder()

X = encoder.transform(tovec_list)

print(X.shape)
print(np.isnan(X).any(), np.isinf(X).any())
print("row norms:", np.linalg.norm(X, axis=1))
print("OOV rate:", encoder.oov_rate(tovec_list, encoder.wv))
# (จำนวนข่าว, 300

(2, 300)
False False
row norms: [1.2088127 1.409333 ]
OOV rate: 0.02956989247311828
